---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [3]:
import pandas as pd
import numpy as np


def blight_model():
    
    
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import roc_auc_score
    from sklearn.model_selection import RandomizedSearchCV

    
    # raw read
    
    
    df_train = pd.read_csv('train.csv', encoding="ISO-8859-1", low_memory=False)
    df_train_country = df_train.copy()
    df_test = pd.read_csv('test.csv', encoding="ISO-8859-1", low_memory=False)
    df_addresses = pd.read_csv('addresses.csv', encoding="ISO-8859-1", low_memory=False)
    df_latlons = pd.read_csv('latlons.csv', encoding="ISO-8859-1", low_memory=False)
    s_id = df_test['ticket_id'].copy().values

    
    # filter all data for Detroit only and link ticket_id to geospatial data (latitude(lat),longitude(lon))
    
    
    df_train = df_train.loc[(df_train['city'] == 'DETROIT') | (df_train['city'] == 'Detroit')]
    df_train = df_train.drop(['violation_zip_code', 'grafitti_status', 'non_us_str_code'], axis=1)
    df_train['compliance'] = df_train['compliance'].fillna(0)
    df_train = df_train.drop([col for col in df_train.columns if col not in df_test.columns and col != 'compliance'], axis=1)
    address2lat_dict = dict(zip(df_latlons['address'], df_latlons['lat']))
    address2lon_dict = dict(zip(df_latlons['address'], df_latlons['lon']))
    id2address_dict = dict(zip(df_addresses['ticket_id'], df_addresses['address']))
    df_train['lat'] = df_train['ticket_id'].apply(lambda x: [address2lat_dict[id2address_dict[x]]][0])
    df_train['lon'] = df_train['ticket_id'].apply(lambda x: [address2lon_dict[id2address_dict[x]]][0])
    df_test['lat'] = df_test['ticket_id'].apply(lambda x: [address2lat_dict[id2address_dict[x]]][0])
    df_test['lon'] = df_test['ticket_id'].apply(lambda x: [address2lon_dict[id2address_dict[x]]][0])

    
    # feature engineering 1
    # create a 'mean_dist' metric. just a hypothesis that high concetrations of violations are a sign of 
    # deeper seated issues in the area that are likely to negatively influence compliance 
    
    
    lat_center = df_train['lat'].mean()
    lon_center = df_train['lon'].mean()
    lat_ind = None
    lon_ind = None
    for i, col in enumerate(df_train.columns):
        if col == 'lat':
            lat_ind = i
        elif col == 'lon':
            lon_ind = i

    def dist(x):
        d = np.sqrt((x[lat_ind] - float(lat_center)) ** 2 + (x[lon_ind] - float(lon_center)) ** 2)
        return x[lat_ind] + x[lon_ind]
    
    df_train['mean_dist'] = df_train.apply(dist, axis=1)
    z = (df_train['mean_dist'] - df_train['mean_dist'].mean()) / (df_train['mean_dist'].std())
    df_train = df_train[(abs(z) < 3)]
    
    
    lat_center = df_test['lat'].mean()
    lon_center = df_test['lon'].mean()
    lat_ind = None
    lon_ind = None
    for i, col in enumerate(df_test.columns):
        if col == 'lat':
            lat_ind = i
        elif col == 'lon':
            lon_ind = i

    def dist_test(x):
        d = np.sqrt((x[lat_ind] - float(lat_center)) ** 2 + (x[lon_ind] - float(lon_center)) ** 2)
        return x[lat_ind] + x[lon_ind]
    df_test['mean_dist'] = df_test.apply(dist_test, axis=1)

    
    # feature engineering 2
    # drop features that are highly correlated with other variables or simply have 
    # too much variation in categorical value that creates too many features (one hot)
    
    
    selective_drop_train = ['ticket_id', 'violator_name', 'violation_street_number', 'mailing_address_str_number'
        , 'state', 'country', 'city', 'violation_description', 'zip_code', 'inspector_name',
                            'mailing_address_str_name', 'lat', 'lon']
    df_train = df_train.drop(selective_drop_train, axis=1)
    selective_drop_test = ['ticket_id', 'violator_name', 'violation_street_number', 'mailing_address_str_number'
        , 'state', 'country', 'city', 'violation_description', 'zip_code', 'inspector_name',
                           'mailing_address_str_name', 'lat', 'lon']
    df_test = df_test.drop(selective_drop_test, axis=1)

    
    # feature engineering 3
    # derive the 'time_to_pay feature'. just based on the hypothesis that the ticket_issued_date itself is not influential
    # from the perspective of the ticket receiver. it becomes useful in the context of the hearing_date. 
    # time_to_pay is hearing_date - ticket_issued_date. the smaller this value, the more urgent it is and that could motivate
    # one way or the other. or, it could be so short a time that it is impossible to pay. or, it is so long a time
    # one forgets to pay. this window of time seems to motivate ones actions more directly than the parent metrics.
    
    
    df_train['hearing_day'] = df_train['hearing_date'].apply(lambda x: str(x).split(' ')[0])
    df_train['hearing_day'] = pd.to_datetime(df_train['hearing_day'])
    df_train['hearing_time'] = df_train['hearing_date'].apply(lambda x: str(x).split(' ')[-1])
    df_train['ticket_issued_day'] = df_train['ticket_issued_date'].apply(lambda x: str(x).split(' ')[0])
    df_train['ticket_issued_day'] = pd.to_datetime(df_train['ticket_issued_day'])
    df_train['ticket_issued_time'] = df_train['ticket_issued_date'].apply(lambda x: str(x).split(' ')[-1])
    df_train['time_to_pay'] = (df_train['hearing_day'] - df_train['ticket_issued_day']) / np.timedelta64(1, 'D')
    df_train = df_train.drop(['hearing_date', 'ticket_issued_date', 'ticket_issued_date',
                              'hearing_day', 'ticket_issued_day'], axis=1)

    df_test['hearing_day'] = df_test['hearing_date'].apply(lambda x: str(x).split(' ')[0])
    df_test['hearing_day'] = pd.to_datetime(df_test['hearing_day'])
    df_test['hearing_time'] = df_test['hearing_date'].apply(lambda x: str(x).split(' ')[-1])
    df_test['ticket_issued_day'] = df_test['ticket_issued_date'].apply(lambda x: str(x).split(' ')[0])
    df_test['ticket_issued_day'] = pd.to_datetime(df_test['ticket_issued_day'])
    df_test['ticket_issued_time'] = df_test['ticket_issued_date'].apply(lambda x: str(x).split(' ')[-1])
    df_test['time_to_pay'] = (df_test['hearing_day'] - df_test['ticket_issued_day']) / np.timedelta64(1, 'D')
    df_test = df_test.drop(['hearing_date', 'ticket_issued_date', 'ticket_issued_date',
                            'hearing_day', 'ticket_issued_day'], axis=1)
    df_test = df_test.drop(['violation_zip_code', 'grafitti_status', 'non_us_str_code'], axis=1)

    
    # convert time into numerical quantity
    
    
    df_train['ticket_issued_time'] = df_train['ticket_issued_time'].apply(lambda x: float(x.split(':')[0]) * 60)
    df_train['hearing_time'] = df_train['hearing_time'].apply(lambda x: float(x.split(':')[0]) * 60)
    df_test['ticket_issued_time'] = df_test['ticket_issued_time'].apply(lambda x: float(x.split(':')[0]) * 60)
    df_test['hearing_time'] = df_test['hearing_time'].apply(lambda x: float(x.split(':')[0]) * 60)

    
    # feature engineering 5
    # create dummy variables for the variables that are unavoidably categorical

    
    for col in df_train.columns[df_train.dtypes.eq('object')]:
        df_train = df_train.join(pd.get_dummies(df_train[col], sparse=True).astype(np.int8)).drop([col], axis=1)
    df_train = df_train.fillna(0)
    for col in df_test.columns[df_test.dtypes.eq('object')]:
        df_test = df_test.join(pd.get_dummies(df_test[col], sparse=True).astype(np.int8)).drop([col], axis=1)
    df_test = df_test.fillna(0)

    
    
    # categorical values in the test set may not exist in train set and vica versa. this is a problem because one hot encoding on
    # these values results in inconsistent features between the two.


    for col in df_train.columns:
        if col not in df_test.columns and col != 'compliance':
            df_train = df_train.drop([col], axis=1)
    for col in df_test.columns:
        if col not in df_train.columns:
            df_test = df_test.drop([col], axis=1)
    

    # fit a supervised learning model
    

    # an iterative approach was used to determine the classifier input values. RandomizedSearchCV was first done to
    # determine a good starting point. A more refined tuning was done using GridSearchCV.
    # Number of trees in random forest
    ##n_estimators = [int(x) for x in np.linspace(start=1, stop=1000, num=20)]
    # Number of features to consider at every split
    ##max_features = ['auto', 'None']
    # Maximum number of levels in tree
    ##max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
    ##max_depth.append(None)
    # Minimum number of samples required to split a node
    ##min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    ##min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    ##bootstrap = [True, False]  # Create the random grid
    ##random_grid = {'n_estimators': n_estimators,
    ##               'max_features': max_features,
    ##               'max_depth': max_depth,
    ##              'min_samples_split': min_samples_split,
    ##             'min_samples_leaf': min_samples_leaf,
    ##             'bootstrap': bootstrap}

    # Use the random grid to search for best hyperparameters
    # First create the base model to tune
    ##rf = RandomForestClassifier()
    # Random search of parameters, using 3 fold cross validation,
    # search across 100 different combinations, and use all available cores
    ##rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=2, cv=5, verbose=2,
    ##                               random_state=42, n_jobs=-1)  # Fit the random search model
    ##X_train, X_test, y_train, y_test = train_test_split(df_train.drop(['compliance'], axis=1),
    ##                                                    df_train['compliance'], random_state=0)
    ##rf_random.fit(X_train, y_train)
    ##print(rf_random.best_params_)
    
    
    clf = RandomForestClassifier(n_estimators=200, min_samples_split=10, 
                                 min_samples_leaf=2, max_features='sqrt', max_depth=50, bootstrap=True)
      
     
    # use most important features due to memory constraints. important list comes from a full run feature_importances_
    # output. features with importance < 1e-3 were eliminated. the following 38 remain
    
    important = ['discount_amount', 'late_fee', 'judgment_amount','Responsible by Admission',
                 'Responsible by Determination','Responsible by Default','state_fee', 'admin_fee',
                 'time_to_pay','fine_amount','Responsible (Fine Waived) by Deter', 'mean_dist',
                 'ticket_issued_time', 'hearing_time', '9-1-36(a)', '9-1-81(a)', 
                 'Buildings, Safety Engineering & Env Department', 'Department of Public Works', '22-2-61',
                 'RIVARD', 'LAWTON', '9-1-104', 'GRATIOT', 'BUTTERNUT', '22-2-88', 'CHRYSLER', '22-2-45',
                 '9-1-110(a)', 'LELAND','AVIS', '9-1-103(C)','Detroit Police Department',
                 '22-2-22','FENKELL', 'MCNICHOLS', 'MT ELLIOTT']

    for col in df_train.columns:
        if col not in important and col != 'compliance':
            df_train = df_train.drop([col], axis=1)
            df_test = df_test.drop([col], axis=1)
    
    
    # submit
    
    clf = clf.fit(df_train.drop(['compliance'],axis=1), df_train['compliance'])
    pred = clf.predict_proba(df_test)
    pred = [val[1]for val in pred]
    pred_series = pd.Series(pred, index = s_id, dtype='float32').rename('compliance')
    pred_series.index.names = ['ticket_id']

    return pred_series


# print(sorted(blight_model()[0:100], reverse=True))

In [4]:
blight_model()

ticket_id
284932    0.115305
285362    0.001119
285361    0.034142
285338    0.021368
285346    0.093853
285345    0.033550
285347    0.062447
285342    0.869849
285530    0.016471
284989    0.008969
285344    0.079182
285343    0.002200
285340    0.001639
285341    0.098277
285349    0.097492
285348    0.031050
284991    0.006469
285532    0.004651
285406    0.002270
285001    0.014890
285006    0.001471
285405    0.002244
285337    0.006808
285496    0.101211
285497    0.057789
285378    0.001603
285589    0.003078
285585    0.085308
285501    0.066570
285581    0.001536
            ...   
376367    0.054459
376366    0.072526
376362    0.236155
376363    0.194158
376365    0.054459
376364    0.072526
376228    0.078721
376265    0.048798
376286    0.109094
376320    0.040464
376314    0.010376
376327    0.284198
376385    0.245765
376435    0.689857
376370    0.900741
376434    0.136069
376459    0.086830
376478    0.003463
376473    0.055045
376484    0.030697
376482    0.002092
37